# Building an Agent

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Courses/Ryerson/8008/Sesssion10')

## Importing necessary libraries

In [ ]:
import pandas as pd
import json
import duckdb
from pydantic import BaseModel, Field
from IPython.display import Markdown

## Defining the tools

it contains the transaction data.

In [ ]:
DATA = 'data/Store_Sales_Price_Elasticity_Promotions_Data.parquet'

In [ ]:
df=pd.read_parquet(DATA)
df.

,Store_Number,SKU_Coded,Product_Class_Code,Sold_Date,Qty_Sold,Total_Sale_Value,On_Promo
0,1320,6172800,22875,2021-11-02,3,56.849998,0
1,2310,6172800,22875,2021-11-03,1,18.950001,0
2,3080,6172800,22875,2021-11-03,1,18.950001,0
3,2310,6172800,22875,2021-11-06,1,18.950001,0
4,4840,6172800,22875,2021-11-07,1,18.950001,0


Creates the SQL table from a local file, if not already done.

In [ ]:
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

In [ ]:
!pip install groq

In [ ]:
# USE your own key from groq
os.environ['GROQ_API_KEY'] = '?????????'
MODEL="llama-3.3-70b-versatile"
import os
from groq import Groq
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


q="What is the temperature ?"
context="it is 25 degree today"

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Query:{q} Context :{context}",
        }
    ],
    model="llama-3.3-70b-versatile",
    #model="deepseek-r1-distill-llama-70b"
  )

pred=chat_completion.choices[0].message.content
print(pred)

The temperature is 25 degrees today.


In [ ]:
def generate_sql_query(prompt: str, columns: list, table_name: str) -> str:
    """Generate an SQL query based on a prompt"""
    formatted_prompt = SQL_GENERATION_PROMPT.format(prompt=prompt,
                                                    columns=columns,
                                                    table_name=table_name)
    print(formatted_prompt)
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": formatted_prompt}],
        model=MODEL,
    )

    return response.choices[0].message.content

In [ ]:
def lookup_sales_data(prompt: str) -> str:
    """Implementation of sales data lookup from parquet file using SQL"""
    try:

        # define the table name
        table_name = "sales"

        # step 1: read the parquet file into a DuckDB table
        df = pd.read_parquet(DATA)
        duckdb.sql(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

        # step 2: generate the SQL code
        sql_query = generate_sql_query(prompt, df.columns, table_name)
        print(sql_query)
        # clean the response to make sure it only includes the SQL code
        sql_query = sql_query.strip()
        sql_query = sql_query.replace("```sql", "").replace("```", "")

        # step 3: execute the SQL query
        result = duckdb.sql(sql_query).df()

        return result.to_string()
    except Exception as e:
        return f"Error accessing data: {str(e)}"

In [ ]:
#example_data = lookup_sales_data("Show me all the sales for store 4840 on November 1st sort by date ")
example_data = lookup_sales_data("List Stores (Store_number) and their count ")


print(example_data)


Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: List Stores (Store_number) and their count 

The available columns are: Index(['Store_Number', 'SKU_Coded', 'Product_Class_Code', 'Sold_Date',
       'Qty_Sold', 'Total_Sale_Value', 'On_Promo'],
      dtype='object')
The table name is: sales

```sql
SELECT Store_Number, COUNT(*) 
FROM sales 
GROUP BY Store_Number;
```
    Store_Number  count_star()
0           1320         30131
1           3080         25225
2           4840         21614
3            330         21452
4           2090         16937
5           1650         29313
6            880         22780
7           1210         27725
8           1760         19642
9           2640         17496
10          3190         17019
11           550         13824
12          2750         24149
13          1980         13898
14          1100         25636
15          4070         17265
16          1540         23858
17          473

### Tool 2: Data Analysis

The second tool can analyze the returned data and display conclusions to users.

<img src="images/tool2.png" width="300"/>

In [ ]:
DATA_ANALYSIS_PROMPT = """
Analyze the following data: {data}
Your job is to answer the following question: {prompt}
"""

In [ ]:
# code for tool 2
def analyze_sales_data(prompt: str, data: str) -> str:
    """Implementation of AI-powered sales data analysis"""
    formatted_prompt = DATA_ANALYSIS_PROMPT.format(data=data, prompt=prompt)
    print(formatted_prompt)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    analysis = response.choices[0].message.content
    return analysis if analysis else "No analysis could be generated"

This tool is relatively simple, but let's still test it out to be sure things are working correctly.

In [ ]:
print(analyze_sales_data(prompt="what trends do you see in this data",
                         data=example_data))


Analyze the following data:     Store_Number  count_star()
0           1320         30131
1           3080         25225
2           4840         21614
3            330         21452
4           2090         16937
5           1650         29313
6            880         22780
7           1210         27725
8           1760         19642
9           2640         17496
10          3190         17019
11           550         13824
12          2750         24149
13          1980         13898
14          1100         25636
15          4070         17265
16          1540         23858
17          4730         13215
18          4180         14725
19          3300         30084
20          2310         22593
21          2200         20083
22          2420         21457
23          2970         35401
24          3410         21544
25          3630         21130
26          1870         22112
27           990         19745
28          3740         18243
29           660         19188
30        